In [6]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
import html5lib

In [34]:
scrip_codes = pd.read_csv("C:\\Users\\garya\\ML projects\\Stock analysis\\datasets\\nsecompanies.csv")
scrip_codes = scrip_codes.drop_duplicates()

In [38]:
len(scrip_codes)

2178

In [39]:
scrip_codes

,ScripCodes
0,20MICRONS
1,21STCENMGM
2,364D030823
3,364D161123
4,3IINFOLTD
...,...
2294,ZOTA
2295,ZUARI
2296,ZUARIIND
2297,ZYDUSLIFE


In [40]:
df = pd.DataFrame()
frames = []

In [41]:
print(scrip_codes)

      ScripCodes
0      20MICRONS
1     21STCENMGM
2     364D030823
3     364D161123
4      3IINFOLTD
...          ...
2294        ZOTA
2295       ZUARI
2296    ZUARIIND
2297   ZYDUSLIFE
2298   ZYDUSWELL

[2178 rows x 1 columns]


In [42]:
def extractNum(s):
    ans = ""
    flag = False
    for c in s:
        if(flag and c == '.'):
            ans += c
            continue
        if(c >= '0' and c<='9'):
            flag = True
        if(flag and c != '\n' and c!=' '):
            ans += c
    return ans

In [43]:
def findRatios(ratios):
    lst = []
    for sibling in ratios:
        lst.append(sibling.text)
    roce = extractNum(lst[13])
    peratio = extractNum(lst[7])
    return roce , peratio

In [44]:
for scrip in scrip_codes['ScripCodes']: 
    URL = f"https://www.screener.in/company/{scrip}/consolidated/"
    status_code = -1 
    html_text = ""
    print(scrip)
    while(status_code != 200):
        html_text = requests.get(URL)
        status_code = html_text.status_code
        print(status_code)
        if(status_code == 404):
            break
    try:
        soup = BeautifulSoup(html_text.text, 'lxml')
        quarter_results = soup.find_all(id = 'quarters')
        yearly_results = soup.find_all(id = 'profit-loss')
        ratios = soup.find(id='top-ratios')
        roce , peratio = findRatios(ratios)
        df_quarter = pd.read_html(str(quarter_results))[0]
        flag = 0
        if(df_quarter.iloc[0,0] == "Revenue"):
            flag  = 1
        elif(df_quarter.iloc[0,0] != "Sales\xa0+"):
            continue
        if(len(df_quarter.columns) == 14 and df_quarter.columns[1] == 'Sep 2019'):
            
            data = {
                "Scrip_Name" : scrip,
                "ROCE" : roce, 
                "P/E Ratio": peratio
            }
            for quarter in df_quarter.columns:
                data.__setitem__(f"Revenue {quarter}", df_quarter[quarter][0])
                data.__setitem__(f"EBITDA {quarter}", df_quarter[quarter][2+flag])
                data.__setitem__(f"Profit {quarter}", df_quarter[quarter][9])
                data.__setitem__(f"Margin {quarter}", df_quarter[quarter][3+flag])
            temp_df = pd.DataFrame(data,index=[1])
            frames.append(temp_df)
    except Exception as e:
        print(scrip, e)

20MICRONS
200
21STCENMGM
200
364D030823
404
364D030823 'NoneType' object is not iterable
364D161123
404
364D161123 'NoneType' object is not iterable
3IINFOLTD
200
3MINDIA
200
3PLAND
200
426GS2023
404
426GS2023 'NoneType' object is not iterable
4THDIM
200
5PAISA
200
63MOONS
200
667GS2035
404
667GS2035 'NoneType' object is not iterable
667GS2050
404
667GS2050 'NoneType' object is not iterable
669GS2024
404
669GS2024 'NoneType' object is not iterable
699GS2051
404
699GS2051 'NoneType' object is not iterable
710GS2029
404
710GS2029 'NoneType' object is not iterable
719GS2060
404
719GS2060 'NoneType' object is not iterable
726GS2032
404
726GS2032 'NoneType' object is not iterable
738GS2027
404
738GS2027 'NoneType' object is not iterable
74GS2062
404
74GS2062 'NoneType' object is not iterable
754GS2036
404
754GS2036 'NoneType' object is not iterable
75GS2034
404
75GS2034 'NoneType' object is not iterable
91D151222
404
91D151222 'NoneType' object is not iterable
A2ZINFRA
200
AAKASH
200
AAREYD

429
200
CGPOWER
429
429
200
CHALET
429
429
429
200
CHAMBLFERT
429
429
429
200
CHEMBOND
429
429
429
200
CHEMCON
429
429
429
200
CHEMFAB
429
429
200
CHEMPLASTS
429
429
429
200
CHENNPETRO
429
429
429
429
200
CHEVIOT
429
429
429
200
CHOICEIN
429
429
429
200
CHOLAFIN
429
429
200
CHOLAHLDNG
429
429
429
200
CIGNITITEC
429
429
200
CINELINE
429
429
200
CINEVISTA
429
429
200
CIPLA
429
429
429
200
CLEAN
429
429
200
CLEDUCATE
429
429
200
CLNINDIA
429
429
429
200
CLOUD
429
429
200
CLSEL
429
429
429
200
CMICABLES
429
429
200
CMMIPL
200
CMSINFO
429
429
200
COALINDIA
429
429
429
200
COASTCORP
429
200
COASTPP
429
429
200
COCHINSHIP
429
429
429
200
COFFEEDAY
429
429
429
200
COFORGE
429
429
429
200
COLPAL
429
429
429
429
200
COMPINFO
429
429
200
COMPUSOFT
429
200
CONCOR
429
429
429
200
CONFIPET
429
429
429
200
CONSOFINVT
429
429
200
CONSUMBEES
429
429
429
429
200
CONTI
429
429
429
200
CONTROLPR
429
429
429
429
200
COOLCAPS
429
429
200
CORALFINAC
429
429
429
200
CORDSCABLE
429
429
200
COROMANDEL
429
429
2

429
429
429
200
HARSHA
429
429
429
200
HATHWAY
429
429
429
200
HATSUN
429
429
429
200
HAVELLS
429
429
200
HAVISHA
429
429
429
200
HBANKETF
429
429
200
HBLPOWER
429
429
429
200
HBSL
200
HCC
429
200
HCG
429
429
429
200
HCL-INSYS
429
429
429
200
HCLTECH
429
429
429
200
HDFC
429
429
429
200
HDFCAMC
429
429
200
HDFCBANK
429
429
429
200
HDFCGROWTH
429
429
200
HDFCLIFE
429
429
429
429
200
HDFCLOWVOL
429
429
429
200
HDFCMFGETF
429
429
429
200
HDFCMOMENT
429
429
200
HDFCNEXT50
200
HDFCNIF100
429
429
429
200
HDFCNIFETF
429
429
429
200
HDFCQUAL
429
429
429
429
200
HDFCSENETF
429
429
200
HDFCSILVER
429
429
200
HDFCVALUE
429
429
200
HDIL
429
429
200
HEADSUP
429
429
429
200
HEALTHY
429
429
429
200
HECPROJECT
429
429
200
HEG
429
429
429
429
200
HEIDELBERG
429
429
429
200
HEMIPROP
429
429
429
200
HERANBA
429
429
429
200
HERCULES
429
429
429
429
200
HERITGFOOD
429
429
429
200
HEROMOTOCO
429
429
429
200
HESTERBIO
429
429
429
200
HEXATRADEX
429
429
429
200
HFCL
429
429
200
HGINFRA
429
429
429
200
HGS
429

429
200
LAKPRE
429
429
200
LALPATHLAB
429
429
429
200
LAMBODHARA
429
429
429
200
LANCER
429
429
429
200
LAOPALA
429
429
429
200
LASA
429
429
429
200
LATENTVIEW
429
429
429
200
LATTEYS
429
429
429
200
LAURUSLABS
429
429
200
LAXMICOT
429
429
429
200
LAXMIMACH
429
429
200
LCCINFOTEC
429
429
429
200
LEMERITE
429
429
200
LEMONTREE
429
429
429
429
200
LFIC
429
429
200
LGBBROSLTD
429
429
429
200
LGBFORGE
429
429
429
200
LIBAS
429
429
429
200
LIBERTSHOE
429
429
200
LICHSGFIN
429
429
429
429
200
LICI
429
200
LICNETFGSC
429
429
429
200
LICNETFN50
200
LICNETFSEN
429
429
200
LICNFNHGP
429
429
429
200
LIKHITHA
429
429
429
429
200
LINC
429
429
200
LINCOLN
429
429
429
200
LINDEINDIA
429
429
429
200
LIQUIDBEES
429
429
429
200
LIQUIDETF
429
429
429
200
LLOYDS
429
200
LODHA
429
429
200
LOKESHMACH
429
429
429
200
LOTUSEYE
429
429
200
LOVABLE
429
429
200
LOYALTEX
429
429
200
LPDC
429
429
429
200
LSIL
429
429
429
200
LT
200
LTGILTBEES
429
429
200
LTI
429
429
429
200
LTTS
429
429
200
LUMAXIND
200
LUMAXTECH


200
POWERGRID
429
429
429
200
POWERINDIA
429
200
POWERMECH
429
429
429
200
PPAP
429
200
PPL
429
429
429
200
PPLPHARMA
429
429
429
200
PRAENG
429
429
429
200
PRAJIND
429
429
429
200
PRAKASH
429
429
200
PRAKASHSTL
429
200
PRAXIS
429
200
PRECAM
429
200
PRECISION
429
429
429
200
PRECOT
429
429
429
200
PRECWIRE
429
429
200
PREMEXPLN
429
429
429
200
PREMIER
429
429
429
200
PREMIERPOL
429
429
200
PRESSMN
429
429
200
PRESTIGE
429
429
429
200
PRICOLLTD
429
429
429
200
PRIMESECU
429
200
PRINCEPIPE
429
429
200
PRITI
429
429
429
200
PRITIKAUTO
429
200
PRIVISCL
200
PROPEQUITY
429
429
200
PROZONINTU
429
429
200
PRSMJOHNSN
429
200
PRUDENT
200
PSB
429
200
PSPPROJECT
429
429
429
200
PSUBNKBEES
429
429
429
200
PTC
429
200
PTL
200
PULZ
429
429
429
429
200
PUNJABCHEM
429
429
429
200
PURVA
429
200
PVP
429
429
200
PVR
429
429
429
200
QGOLDHALF
429
429
429
200
QMSMEDI
429
429
429
200
QNIFTY
200
QUADPRO
429
429
200
QUESS
429
429
429
200
QUICKHEAL
429
429
429
200
RADAAN
429
429
429
200
RADHIKAJWE
429
429
429
2

429
429
200
SHAREINDIA
429
429
429
200
SHARIABEES
429
429
200
SHEMAROO
429
429
429
200
SHIGAN
429
429
429
200
SHILPAMED
429
429
429
200
SHIVALIK
429
429
200
SHIVAMAUTO
429
429
429
200
SHIVAMILLS
429
429
200
SHIVATEX
429
429
429
429
200
SHK
429
429
429
200
SHOPERSTOP
429
429
429
200
SHRADHA
429
429
429
200
SHREDIGCEM
429
429
429
200
SHREECEM
429
429
429
200
SHREEPUSHK
429
429
429
200
SHREERAMA
429
200
SHRENIK
429
429
200
SHREYANIND
429
429
429
200
SHREYAS
429
429
429
429
200
SHRIPISTON
429
200
SHRIRAMCIT
429
429
200
SHRIRAMPPS
429
429
200
SHYAMCENT
429
429
429
429
200
SHYAMMETL
429
429
200
SHYAMTEL
429
429
429
200
SIEMENS
429
429
200
SIGACHI
429
429
429
200
SIGIND
429
429
200
SIGMA
429
429
200
SIKKO
429
429
429
200
SIL
429
429
429
200
SILGO
429
429
429
200
SILINV
429
429
429
200
SILLYMONKS
429
429
200
SILVER
429
429
429
200
SILVERBEES
429
429
429
200
SILVERTUC
429
429
429
200
SIMBHALS
429
200
SIMPLEXINF
429
429
429
200
SINTERCOM
429
429
200
SIRCA
429
429
429
200
SIS
429
429
429
200
SITI

429
200
VISAKAIND
429
429
429
200
VISASTEEL
429
429
200
VISESHINFO
429
200
VISHAL
429
429
429
200
VISHNU
429
429
200
VISHWARAJ
429
429
429
200
VITAL
429
429
429
200
VIVIANA
429
429
429
200
VIVIDHA
429
429
429
200
VIVIMEDLAB
429
429
429
200
VIVO
429
429
429
200
VLSFINANCE
429
429
429
200
VMARCIND
429
429
429
200
VMART
429
429
429
200
VOLTAMP
429
429
429
429
200
VOLTAS
429
429
429
200
VRLLOG
429
429
200
VSSL
429
429
429
200
VSTIND
429
429
429
200
VSTTILLERS
429
429
200
VTL
429
429
429
429
200
WABAG
429
429
200
WALCHANNAG
429
429
200
WANBURY
429
429
429
200
WATERBASE
429
429
429
200
WEALTH
429
429
429
429
200
WEBELSOLAR
429
429
200
WEIZMANIND
429
429
429
429
200
WELCORP
429
429
429
200
WELENT
429
429
429
200
WELINV
429
429
200
WELSPUNIND
429
429
429
429
200
WENDT
429
429
200
WESTLIFE
429
429
429
429
200
WEWIN
429
429
200
WFL
429
429
200
WHEELS
429
429
429
200
WHIRLPOOL
429
429
429
200
WILLAMAGOR
429
429
429
200
WINDLAS
429
429
429
200
WINDMACHIN
429
429
429
200
WINPRO
429
429
429
200
WIPL

In [45]:
df = pd.concat(frames, ignore_index=True)
# df = df.dropna()

In [46]:
print(df)

      Scrip_Name   ROCE P/E Ratio Revenue Unnamed: 0 EBITDA Unnamed: 0  \
0      20MICRONS  19.5%      6.93            Sales +  Operating Profit   
1     21STCENMGM  53.7%                      Sales +  Operating Profit   
2      3IINFOLTD  2.35%       103            Sales +  Operating Profit   
3        3MINDIA  17.8%      71.6            Sales +  Operating Profit   
4         3PLAND  2.77%      22.0            Sales +  Operating Profit   
...          ...    ...       ...                ...               ...   
1137        ZOTA  14.9%       272            Sales +  Operating Profit   
1138       ZUARI  15.7%      13.9            Sales +  Operating Profit   
1139    ZUARIIND  3.75%      2.21            Sales +  Operating Profit   
1140   ZYDUSLIFE  14.7%      21.2            Sales +  Operating Profit   
1141   ZYDUSWELL  6.41%      33.4            Sales +  Operating Profit   

     Profit Unnamed: 0 Margin Unnamed: 0 Revenue Sep 2019 EBITDA Sep 2019  \
0           Net Profit            

In [50]:
df.to_excel(f"C:\\Users\\garya\\stock market telegram\\dataset\\previous13QuartersData.xlsx", index=False)